<a href="https://colab.research.google.com/github/ApiLogistica3sem/ApiLogistica3sem/blob/main/Otimiza%C3%A7%C3%A3o_ATT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#OTIMIZAÇÃO
from pulp import *

# INSERIR CAPACIDADES DAS FÁBRICAS
capacidades = [16000000, 9500000, 16500000]

# DEMANDAS DOS CLIENTES (extraídas do CSV)
demandas = [
    1021200, 757500, 343200, 782700, 1969500, 1194600, 1151400, 1345200,
    2634600, 821400, 2973000, 508200, 4500, 14400, 472800, 371100, 11700,
    743400, 519300, 8400, 12600, 498600, 6000, 848100, 6900, 1423200,
    761400, 1548300, 438600, 594000, 979500, 17700, 9600, 637800, 988800,
    660300, 1256700, 951900, 21600, 1484100, 1168800, 1414500, 1524900,
    1039500, 856800, 25200, 1461600, 1184400, 18900, 1544400, 21300
]

# CUSTOS PARA ENTREGAS NOS CLIENTES
custos = [
    [0.57, 0.56, 0.37, 0.49, 0.54, 0.58, 0.59, 0.58, 0.46, 0.7,
     0.41, 0.77, 0.82, 0.85, 0.81, 0.82, 0.91, 0.84, 0.71, 0.75,
     0.78, 0.72, 0.74, 0.73, 0.71, 0.71, 0.71, 0.76, 0.81, 0.86,
     0.74, 0.81, 0.76, 0.7, 0.73, 0.0, 0.0, 0.0, 0.82, 0.88,
     0.88, 0.9, 0.91, 0.61, 0.61, 0.65, 0.42, 0.33, 0.34, 0.38, 0.37],
    [0.0, 0.0, 0.0, 0.0, 0.38, 0.0, 0.0, 0.39, 0.41, 0.35,
     0.38, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
     0.0, 0.0, 0.0, 0.4, 0.0, 0.39, 0.34, 0.35, 0.37, 0.49,
     0.39, 0.41, 0.66, 0.41, 0.41, 0.5, 0.5, 0.48, 0.58, 0.57,
     0.59, 0.6, 0.57, 0.0, 0.0, 0.0, 0.73, 0.74, 0.73, 0.71, 0.73],
    [0.47, 0.45, 0.41, 0.46, 0.48, 0.42, 0.48, 0.42, 0.5, 0.4,
     0.58, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.41, 0.35,
     0.33, 0.34, 0.0, 0.37, 0.0, 0.34, 0.32, 0.41, 0.25, 0.0,
     0.36, 0.33, 0.34, 0.34, 0.36, 0.71, 0.73, 0.7, 0.0, 0.0,
     0.0, 0.0, 0.0, 0.49, 0.49, 0.55, 0.88, 0.89, 1.0, 0.92, 1.24]
]

# NUMERO DE LINHAS EM CADA PARAMETRO
num_fabricas = len(capacidades)
num_clientes = len(demandas)

#FORMULAÇÃO DO PROBLEMA
transp = LpProblem('MinTransporte', LpMinimize)

x = {(i, j): LpVariable(f'x{i}_{j}', lowBound=0, cat=LpInteger) for i in range(1, num_fabricas + 1) for j in range(1, num_clientes + 1)}

transp += lpSum(custos[i-1][j-1] * x[i, j] for i in range(1, num_fabricas + 1) for j in range(1, num_clientes + 1))

for i in range(1, num_fabricas + 1):
    transp += lpSum(x[i, j] for j in range(1, num_clientes + 1)) <= capacidades[i-1]

for j in range(1, num_clientes + 1):
    transp += lpSum(x[i, j] for i in range(1, num_fabricas + 1)) == demandas[j-1]

transp.solve()

print("Quantidade a ser produzida:", LpStatus[transp.status])
for v in transp.variables():
    print(f"{v.name} = {v.varValue}")

print("Custo Total = ", value(transp.objective))

import csv

# Mapear os códigos das fábricas e clientes
fabricas = {1: 3403208, 2: 3423909, 3: 3424402}
clientes = [
    2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 2312, 2313, 2314, 2315,
    2316, 2317, 2318, 2319, 2320, 2321, 2322, 2323, 2324, 2325, 2326, 2327, 2328, 2329, 2330,
    2331, 2332, 2333, 2334, 2335, 2336, 2337, 2338, 2339, 2340, 2341, 2342, 2343, 2344, 2345,
    2346, 2347, 2348, 2349, 2350, 2351
]

# Criar uma lista com os resultados da otimização
resultados = [["Fabrica (CO.Fabrica)", "Cliente (CO.Cliente)", "Quantidade"]]
for (i, j), var in x.items():
    if var.varValue > 0:
        fab = fabricas[i]  # Fábrica (CO.Fabrica)
        cli = clientes[j-1]  # Cliente (CO.Cliente), ajustando o índice
        resultados.append([fab, cli, var.varValue])

# Nome do arquivo
nome_arquivo = "Otimização.csv"

# Salvar resultados em um arquivo CSV
with open(nome_arquivo, mode="w", newline="") as arquivo:
    escritor = csv.writer(arquivo)
    escritor.writerows(resultados)

# Baixar automaticamente o arquivo (para Google Colab)
try:
    from google.colab import files
    files.download(nome_arquivo)
except ImportError:
    print(f"O arquivo {nome_arquivo} foi salvo no diretório local.")

# Exibir a resposta no terminal
print("Resultado da otimização:")
for linha in resultados:
    print(f"Fábrica {linha[0]} (CO.Fabrica) - Cliente {linha[1]} (CO.Cliente) - Quantidade {linha[2]}")


Quantidade a ser produzida: Optimal
x1_1 = 0.0
x1_10 = 0.0
x1_11 = 2973000.0
x1_12 = 0.0
x1_13 = 0.0
x1_14 = 0.0
x1_15 = 0.0
x1_16 = 0.0
x1_17 = 0.0
x1_18 = 0.0
x1_19 = 0.0
x1_2 = 0.0
x1_20 = 0.0
x1_21 = 0.0
x1_22 = 0.0
x1_23 = 0.0
x1_24 = 0.0
x1_25 = 0.0
x1_26 = 0.0
x1_27 = 0.0
x1_28 = 0.0
x1_29 = 0.0
x1_3 = 0.0
x1_30 = 0.0
x1_31 = 0.0
x1_32 = 0.0
x1_33 = 0.0
x1_34 = 0.0
x1_35 = 0.0
x1_36 = 660300.0
x1_37 = 1256700.0
x1_38 = 951900.0
x1_39 = 0.0
x1_4 = 0.0
x1_40 = 0.0
x1_41 = 0.0
x1_42 = 0.0
x1_43 = 0.0
x1_44 = 0.0
x1_45 = 0.0
x1_46 = 0.0
x1_47 = 1461600.0
x1_48 = 1184400.0
x1_49 = 18900.0
x1_5 = 1969500.0
x1_50 = 1544400.0
x1_51 = 21300.0
x1_6 = 0.0
x1_7 = 0.0
x1_8 = 377500.0
x1_9 = 2634600.0
x2_1 = 1021200.0
x2_10 = 0.0
x2_11 = 0.0
x2_12 = 0.0
x2_13 = 0.0
x2_14 = 0.0
x2_15 = 0.0
x2_16 = 0.0
x2_17 = 0.0
x2_18 = 0.0
x2_19 = 519300.0
x2_2 = 757500.0
x2_20 = 8400.0
x2_21 = 12600.0
x2_22 = 498600.0
x2_23 = 0.0
x2_24 = 0.0
x2_25 = 0.0
x2_26 = 0.0
x2_27 = 0.0
x2_28 = 1289000.0
x2_29 = 0.0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Resultado da otimização:
Fábrica Fabrica (CO.Fabrica) (CO.Fabrica) - Cliente Cliente (CO.Cliente) (CO.Cliente) - Quantidade Quantidade
Fábrica 3403208 (CO.Fabrica) - Cliente 2305 (CO.Cliente) - Quantidade 1969500.0
Fábrica 3403208 (CO.Fabrica) - Cliente 2308 (CO.Cliente) - Quantidade 377500.0
Fábrica 3403208 (CO.Fabrica) - Cliente 2309 (CO.Cliente) - Quantidade 2634600.0
Fábrica 3403208 (CO.Fabrica) - Cliente 2311 (CO.Cliente) - Quantidade 2973000.0
Fábrica 3403208 (CO.Fabrica) - Cliente 2336 (CO.Cliente) - Quantidade 660300.0
Fábrica 3403208 (CO.Fabrica) - Cliente 2337 (CO.Cliente) - Quantidade 1256700.0
Fábrica 3403208 (CO.Fabrica) - Cliente 2338 (CO.Cliente) - Quantidade 951900.0
Fábrica 3403208 (CO.Fabrica) - Cliente 2347 (CO.Cliente) - Quantidade 1461600.0
Fábrica 3403208 (CO.Fabrica) - Cliente 2348 (CO.Cliente) - Quantidade 1184400.0
Fábrica 3403208 (CO.Fabrica) - Cliente 2349 (CO.Cliente) - Quantidade 18900.0
Fábrica 3403208 (CO.Fabrica) - Cliente 2350 (CO.Cliente) - Quantidade 